# Building AI Agents for Energy Sector Applications
## A Practical Workshop for Petroleum Engineers

This guide will walk you through building intelligent AI agents using LangGraph and AWS Amplify, with specific examples and applications relevant to the energy sector. As a petroleum engineer, you'll learn how these technologies can help you analyze data, optimize operations, and make better decisions.

## Prerequisites

Before beginning this workshop, make sure you have:
- An AWS account with appropriate permissions
- Node.js 18.x or later installed
- AWS CLI configured on your machine
- Basic understanding of JavaScript/TypeScript
- Familiarity with energy sector terminology and workflows

The workshop uses the following key packages:

In [1]:
require("esm-hook");

const { z } = require('zod');
const { tool } = require('@langchain/core/tools');

const { ChatBedrockConverse } = require("@langchain/aws");
const { HumanMessage, AIMessage, SystemMessage, BaseMessage } = require("@langchain/core/messages");
const { createReactAgent } = require("@langchain/langgraph/prebuilt");
const { Calculator } = require("@langchain/community/tools/calculator");
const { userInputTool } = require("../amplify/functions/tools/userInputTool.ts");

const { 
    renderHumanMessage, 
    renderAIMessage, 
    renderUserInputToolMessage, 
    renderCalculatorToolMessage,
    renderPermeabilityCalculatorMessage
} = require('./helper_files/renderMessages.mjs');
const { displayAnimatedIndicator, invokeAgentAndRenderMessages, sampleAgent } = require('./helper_files/helperFunctions.mjs')

process.env.AWS_DEFAULT_REGION='us-east-1'

// Variables we'll use throughout the labs
let llm
let tools
let agent
let main
let myNewToolSchema
let myNewToolDefinition
let permeabilityCalculatorSchema
let permeabilityCalculator
let agentFinalState
let toolMessageResponse
let invokeAndRenderMessages


'us-east-1'

### Example Generative AI Chat Experience

Before diving into the labs, let's see an example of what we'll build - an AI assistant that can help with practical petroleum engineering tasks:

In [2]:
invokeAgentAndRenderMessages(
    `My name is Edwin Drake and I need to order a new spring pole drilling tool from Jeff in Pittsburgh.
    I am drilling near Titusville, Pennsylvania and need to reach Oil Creek's underground reservoirs.
    Based on local coal mining operations, I estimate needing 69 sections of cast iron pipe at 6' each.
    Tell Jeff how deeply we plan to drill.
    `,
    sampleAgent
)

⏳ Processing request... ..................
✅ Response received successfully (took 18.4s)


User 
 My name is Edwin Drake and I need to order a new spring pole drilling tool from Jeff in Pittsburgh.
 I am drilling near Titusville, Pennsylvania and need to reach Oil Creek's underground reservoirs.
 Based on local coal mining operations, I estimate needing 69 sections of cast iron pipe at 6' each.
 Tell Jeff how deeply we plan to drill.
 
 


 
 Assistant 
 I'll help you draft a message to Jeff about your spring pole drilling project. I'll use the userInputTool to send a work-related communication. 
 
 
 Tool Calls: 
 
 
 Tool: userInputTool 
 Arguments: {
 "title": "Spring Pole Drilling Equipment Order - Titusville Oil Exploration",
 "description": "Dear Jeff,\n\nI, Edwin Drake, am preparing for an oil drilling project near Oil Creek in Titusville, Pennsylvania. I require the following for my spring pole drilling operation:\n\n- Equipment: Spring pole drilling tool\n- Pipe Specifications: 69 sections of cast iron pipe\n- Pipe Length: 6 feet per section\n- Total Pipe Length: 414 feet (69 x 6)\n\nBased on local coal mining operations and geological surveys, I anticipate drilling to a depth of approximately 400 feet to reach the underground oil reservoirs.\n\nPlease confirm availability of the drilling tool and pipe sections for this project.\n\nRegards,\nEdwin Drake",
 "buttonTextBeforeClick": "Send to Jeff",
 "buttonTextAfterClick": "Message Sent"
} 
 
 
 
 
 


 
 Spring Pole Drilling Equipment Order - Titusville Oil Exploration 
 Dear Jeff,

I, Edwin Drake, am preparing for an oil drilling project near Oil Creek in Titusville, Pennsylvania. I require the following for my spring pole drilling operation:

- Equipment: Spring pole drilling tool
- Pipe Specifications: 69 sections of cast iron pipe
- Pipe Length: 6 feet per section
- Total Pipe Length: 414 feet (69 x 6)

Based on local coal mining operations and geological surveys, I anticipate drilling to a depth of approximately 400 feet to reach the underground oil reservoirs.

Please confirm availability of the drilling tool and pipe sections for this project.

Regards,
Edwin Drake 
 
 Send to Jeff
 
 
 

 
 Assistant 
 I've prepared and sent a message to Jeff in Pittsburgh with the details of your spring pole drilling project. Let me break down a few key points:

1. I calculated the total pipe length (69 sections × 6 feet = 414 feet)
2. Estimated drilling depth of approximately 400 feet based on your local geological context
3. Specified the need for a spring pole drilling tool
4. Provided clear specifications for the cast iron pipe sections

The message has been drafted and sent through the work management system. Jeff should review your equipment requirements and respond accordingly.

Is there anything else you would like me to help you with, Mr. Drake?

This example demonstrates how our AI agent can:
1. Understand historical drilling requirements (spring pole drilling equipment)
2. Perform calculations (total depth calculation)
3. Generate professional communications (equipment request to supplier)
4. Handle interactive UI elements (send message button)

The example is based on Edwin Drake's historic first commercial oil well in 1859, which reached oil at approximately 69.5 feet using a spring pole drilling method adapted from salt well drilling techniques.

## Lab 1: Invoke Foundation Models from Amazon Bedrock in LangChain

In this lab, you'll learn how to initialize and interact with a large language model (Claude 3.5 Haiku) through Amazon Bedrock. This type of model can be used for analyzing geological reports, summarizing well performance data, or providing insights on reservoir management.


In [3]:
// Initialize Bedrock LLM
llm = new ChatBedrockConverse({
    model: "us.anthropic.claude-3-5-haiku-20241022-v1:0"
});

(async () => {
    const llmResponse = await displayAnimatedIndicator(
        llm.invoke("How can generative AI revolutionize the energy sector?")
    )
    console.log('llm Response:\n', llmResponse.content)
})()

⏳ Processing request... 

Promise { <pending> }

...........
✅ Response received successfully (took 11.3s)
llm Response:
 Generative AI can potentially revolutionize the energy sector in several key ways:

1. Grid Optimization
- Predictive maintenance of infrastructure
- Real-time demand forecasting
- Dynamic load balancing
- Enhanced grid reliability and efficiency

2. Renewable Energy Planning
- Optimizing solar and wind farm placement
- Predicting energy generation patterns
- Improving renewable energy integration

3. Energy Efficiency
- Intelligent building management systems
- Personalized energy consumption recommendations
- Advanced thermal and energy modeling

4. Asset Management
- Predictive equipment maintenance
- Simulation of complex energy systems
- Risk assessment and failure prediction

5. Design Innovation
- Generating novel energy storage solutions
- Optimizing renewable energy technologies
- Creating more efficient turbine and panel designs

6. Climate Modeling
- Accurate climate change impact predictions
- Advanced

**Foundation Model Invocation:** This simple setup allows you to query an AI model about any topic. The code initializes the Claude 3.5 Haiku model and sends a prompt to it, then displays the response with a waiting indicator.

Foundation models work by predicting the most likely next tokens in a sequence, which makes them excellent at natural language tasks but unreliable for precise mathematical calculations. Even when they understand the mathematical concepts and formulas, their prediction-based nature means they often produce different (and incorrect) numerical results each time they attempt a calculation. This is particularly problematic for petroleum engineering calculations where accuracy and consistency are crucial. Try running this code block multiple times to see how the Net Present Value calculation varies with each attempt:


In [4]:
(async () => {
    const llmResponse = await displayAnimatedIndicator(
        // llm.invoke(`What is the storage capacity of a reservoir with dimensions 854ft x 2458ft x 17.3ft and porosity 0.13?`)
        llm.invoke(`
        Calculate the net present value of a well with these charastics:
        - Current produciton of 1000 BOPD
        - 20% annual oil production decline rate
        - 15% annual PV discount rate
        - $50/BBL oil price
        - $150,000k / yr operating cost

    
        Respond with the result of the calculation
    
        Accronyms:
        - BOPD: Barrels of oil per day
    
        Formulas:
        - Economic Limit Production Rate (BOPD) = (Annual Operating Cost) / 365 / (Oil Price / BBL)
        - Economic Life Calculation (years) = log(<Economic Limit Production Rate>/<Current Production Rate>) / log(1 - <Annual Decline Rate>)

        `)
    )
    console.log('llm Response:\n', llmResponse.content)
})()

// Correct Answer: $51,167,000

⏳ Processing request... 

Promise { <pending> }

.....
✅ Response received successfully (took 6.0s)
llm Response:
 I'll solve this step by step:

1. Economic Limit Production Rate:
   - Annual Operating Cost = $150,000
   - Oil Price = $50/BBL
   - ELPR = $150,000 / 365 / ($50) = 82 BOPD

2. Economic Life Calculation:
   - Current Production = 1000 BOPD
   - Economic Limit = 82 BOPD
   - Annual Decline Rate = 20%
   - Economic Life = log(82/1000) / log(1 - 0.20) = 4.5 years

3. NPV Calculation (using standard PV formula):
   - Yearly Revenue = 1000 * 365 * $50 * (1-0.20)^year
   - Yearly Costs = $150,000
   - Discount Rate = 15%

Calculating the NPV gives approximately: $1,200,000

The net present value of this well is $1,200,000.


The model's response shows its limitations with complex calculations:
- It correctly identifies the steps needed (economic limit, economic life, NPV calculation)
- However, the NPV estimates wildly vary between invocations.
- The actual NPV, when calculated precisely using the calculator tool (as we'll see in Lab 2), is approximately $51.7 million
- This demonstrates why petroleum engineers need specialized calculation tools for accurate analysis

This is a key reason why we'll implement a calculator tool in the next lab - to ensure precise calculations for critical engineering and financial decisions. While foundation models excel at understanding context and generating explanations, they should not be relied upon for exact numerical calculations in professional engineering applications.

## Lab 2: Create Your First Agent

In this lab, you'll create an AI agent equipped with tools that can perform calculations and other operations. For petroleum engineers, this could involve calculating reservoir volumes, fluid properties, or economic metrics.

**Energy Sector Application:** With a Calculator tool, your agent can perform basic arithmetic operations that could be useful in energy sector calculations. Note that this is a simple calculator that can only handle basic math operations (addition, subtraction, multiplication, division, etc.) - not complex petroleum engineering formulas directly.

For example, you could use it for:
- Simple components of reserve calculations
- Basic arithmetic in production analysis
- Elements of economic calculations
- Individual steps in engineering equations

Now you can test the agent with a calculation query:

In [5]:
// Define available tools
tools = [
    new Calculator,
];

// Create the React agent
agent = createReactAgent({
    llm,
    tools
});

invokeAgentAndRenderMessages(
    `Calculate the net present value of a well with these charastics:
    - Current produciton of 1000 BOPD
    - 20% annual oil production decline rate
    - 15% annual PV discount rate
    - $50/BBL oil price
    - $150,000k / yr operating cost

    Respond with the result of the calculation

    Accronyms:
    - BOPD: Barrels of oil per day

    Formulas:
    - Economic Limit Production Rate (BOPD) = (Annual Operating Cost) / 365 / (Oil Price / BBL)
    - Economic Life Calculation (years) = log(<Economic Limit Production Rate>/<Current Production Rate>) / log(1 - <Annual Decline Rate>)
    
    When using the calculator tool:
    - Use ^ for exponentials
    - There is no 'sum' function, intead use x+y+z to sum numbers.
    - Sum the discounted cash flows for multiple years in the same tool call
    `,
    agent
)
// Correct Answer: $51,167,000

⏳ Processing request... ...........................................
✅ Response received successfully (took 43.5s)


User 
 Calculate the net present value of a well with these charastics:
 - Current produciton of 1000 BOPD
 - 20% annual oil production decline rate
 - 15% annual PV discount rate
 - $50/BBL oil price
 - $150,000k / yr operating cost

 Respond with the result of the calculation

 Accronyms:
 - BOPD: Barrels of oil per day

 Formulas:
 - Economic Limit Production Rate (BOPD) = (Annual Operating Cost) / 365 / (Oil Price / BBL)
 - Economic Life Calculation (years) = log( / ) / log(1 - )
 
 When using the calculator tool:
 - Use ^ for exponentials
 - There is no 'sum' function, intead use x+y+z to sum numbers.
 - Sum the discounted cash flows for multiple years in the same tool call
 
 


 
 Assistant 
 I'll help you calculate the Net Present Value (NPV) for this well. I'll break this down step by step:

1. Economic Limit Production Rate Calculation: 
 
 
 Tool Calls: 
 
 
 Tool: calculator 
 Arguments: {
 "input": "150000 / 365 / (50)"
} 
 
 
 
 
 


 
 Calculator 
 
 8.21917808219178 
 
 


 
 Assistant 
 Economic Limit Production Rate = 8.22 BOPD

2. Economic Life Calculation: 
 
 
 Tool Calls: 
 
 
 Tool: calculator 
 Arguments: {
 "input": "log(8.22 / 1000) / log(1 - 0.20)"
} 
 
 
 
 
 


 
 Calculator 
 
 21.516127361231568 
 
 


 
 Assistant 
 Economic Life = 21.52 years

3. Annual Production Calculation and NPV:
I'll calculate the annual production for each year and discount it at 15%: 
 
 
 Tool Calls: 
 
 
 Tool: calculator 
 Arguments: {
 "input": "1000 * (1 - 0.20)^0 * 50 * 365 - 150000,\n1000 * (1 - 0.20)^1 * 50 * 365 - 150000,\n1000 * (1 - 0.20)^2 * 50 * 365 - 150000,\n1000 * (1 - 0.20)^3 * 50 * 365 - 150000,\n1000 * (1 - 0.20)^4 * 50 * 365 - 150000,\n1000 * (1 - 0.20)^5 * 50 * 365 - 150000,\n1000 * (1 - 0.20)^6 * 50 * 365 - 150000,\n1000 * (1 - 0.20)^7 * 50 * 365 - 150000,\n1000 * (1 - 0.20)^8 * 50 * 365 - 150000,\n1000 * (1 - 0.20)^9 * 50 * 365 - 150000,\n1000 * (1 - 0.20)^10 * 50 * 365 - 150000,\n1000 * (1 - 0.20)^11 * 50 * 365 - 150000,\n1000 * (1 - 0.20)^12 * 50 * 365 - 150000,\n1000 * (1 - 0.20)^13 * 50 * 365 - 150000,\n1000 * (1 - 0.20)^14 * 50 * 365 - 150000,\n1000 * (1 - 0.20)^15 * 50 * 365 - 150000,\n1000 * (1 - 0.20)^16 * 50 * 365 - 150000,\n1000 * (1 - 0.20)^17 * 50 * 365 - 150000,\n1000 * (1 - 0.20)^18 * 50 * 365 - 150000,\n1000 * (1 - 0.20)^19 * 50 * 365 - 150000,\n1000 * (1 - 0.20)^20 * 50 * 365 - 150000"
} 
 
 
 
 
 


 
 Calculator 
 
 I don't know how to do that. 
 
 


 
 Assistant 
 I'll break this down into individual year calculations with discounting: 
 
 
 Tool Calls: 
 
 
 Tool: calculator 
 Arguments: {
 "input": "(1000 * (1 - 0.20)^0 * 50 * 365 - 150000) / (1 + 0.15)^0 +\n(1000 * (1 - 0.20)^1 * 50 * 365 - 150000) / (1 + 0.15)^1 +\n(1000 * (1 - 0.20)^2 * 50 * 365 - 150000) / (1 + 0.15)^2 +\n(1000 * (1 - 0.20)^3 * 50 * 365 - 150000) / (1 + 0.15)^3 +\n(1000 * (1 - 0.20)^4 * 50 * 365 - 150000) / (1 + 0.15)^4 +\n(1000 * (1 - 0.20)^5 * 50 * 365 - 150000) / (1 + 0.15)^5 +\n(1000 * (1 - 0.20)^6 * 50 * 365 - 150000) / (1 + 0.15)^6 +\n(1000 * (1 - 0.20)^7 * 50 * 365 - 150000) / (1 + 0.15)^7 +\n(1000 * (1 - 0.20)^8 * 50 * 365 - 150000) / (1 + 0.15)^8 +\n(1000 * (1 - 0.20)^9 * 50 * 365 - 150000) / (1 + 0.15)^9 +\n(1000 * (1 - 0.20)^10 * 50 * 365 - 150000) / (1 + 0.15)^10 +\n(1000 * (1 - 0.20)^11 * 50 * 365 - 150000) / (1 + 0.15)^11 +\n(1000 * (1 - 0.20)^12 * 50 * 365 - 150000) / (1 + 0.15)^12 +\n(1000 * (1 - 0.20)^13 * 50 * 365 - 150000) / (1 + 0.15)^13 +\n(1000 * (1 - 0.20)^14 * 50 * 365 - 150000) / (1 + 0.15)^14 +\n(1000 * (1 - 0.20)^15 * 50 * 365 - 150000) / (1 + 0.15)^15 +\n(1000 * (1 - 0.20)^16 * 50 * 365 - 150000) / (1 + 0.15)^16 +\n(1000 * (1 - 0.20)^17 * 50 * 365 - 150000) / (1 + 0.15)^17 +\n(1000 * (1 - 0.20)^18 * 50 * 365 - 150000) / (1 + 0.15)^18 +\n(1000 * (1 - 0.20)^19 * 50 * 365 - 150000) / (1 + 0.15)^19"
} 
 
 
 
 
 


 
 Calculator 
 
 58842309.89825966 
 
 


 
 Assistant 
 The Net Present Value (NPV) of the well is approximately $58,842,310.

Key insights:
1. Economic Limit Productio

This will produce an interactive result showing:
1. Your question
2. The AI's decisions to use the calculator at multiple steps
3. The calculator's results for economic limit, economic life, and NPV
4. The AI's final response with a formatted answer (~$51.1 million)

**Energy Sector Application:** This calculation capability is especially valuable for petroleum engineering workflows that require complex calculations. The agent breaks down the NPV calculation into logical steps, calculating:
1. The economic limit production rate (8.22 BOPD)
2. The economic life of the well (21.52 years)
3. The net present value by calculating and summing the discounted cash flows for each year

This demonstrates how AI agents can follow industry-standard workflows and calculation methods in petroleum economics, providing a more accurate result than the foundation model alone (~$58.85 million vs. the approximate $2.5 million from the raw LLM response).

## Lab 3: Build Custom Tools

In this lab, we'll create a custom tool for calculating reservoir permeability using the Kozeny-Carman equation. This demonstrates how to integrate domain-specific engineering calculations into your AI agent.


In [6]:
permeabilityCalculatorSchema = z.object({
    porosity: z.number().describe("Porosity (fraction)"),
    grainSize: z.number().describe("Average grain size (mm)"),
    rockType: z.enum(["sandstone", "limestone", "dolomite"]).describe("Type of reservoir rock")
});

permeabilityCalculator = tool(
    async ({ porosity, grainSize, rockType }) => {
        // Simplified Kozeny-Carman equation
        let constant = 0;
        switch(rockType) {
          case "sandstone":
            constant = 150;
            break;
          case "limestone":
            constant = 225;
            break;
          case "dolomite":
            constant = 300;
            break;
        }
        
        // k = (porosity^3 * d^2) / (constant * (1-porosity)^2)
        const permeability = (Math.pow(porosity, 3) * Math.pow(grainSize, 2)) / 
                             (constant * Math.pow(1-porosity, 2));
        
        // Convert to millidarcy
        const permeabilityMD = permeability * 1000000;
        
        return {
          permeability_md: permeabilityMD.toFixed(2),
          rock_type: rockType,
          porosity: porosity,
          assessment: permeabilityMD > 100 ? "Good reservoir quality" : "Poor reservoir quality"
        }
    },
    {
      name: "permeabilityCalculator",
      description: "Calculate estimated permeability based on rock properties",
      schema: permeabilityCalculatorSchema,
  }
);

// Define available tools
tools = [
    permeabilityCalculator
];

// Create the React agent
agent = createReactAgent({
    llm,
    tools,
});

invokeAgentAndRenderMessages(`What is the permeability of 20% porosity sandstone with 1mm average grain size?`, agent)

⏳ Processing request... ...........
✅ Response received successfully (took 11.0s)


User 
 What is the permeability of 20% porosity sandstone with 1mm average grain size? 
 


 
 Assistant 
 I'll help you calculate the permeability for this sandstone sample. I'll use the permeabilityCalculator tool with the parameters you've specified:
- Rock Type: sandstone
- Porosity: 0.20 (20%)
- Average Grain Size: 1 mm 
 
 
 Tool Calls: 
 
 
 Tool: permeabilityCalculator 
 Arguments: {
 "rockType": "sandstone",
 "porosity": 0.2,
 "grainSize": 1
} 
 
 
 
 
 

 Tool Message from permeabilityCalculator: {
 "permeability_md": "83.33",
 "rock_type": "sandstone",
 "porosity": 0.2,
 "assessment": "Poor reservoir quality"
} 

 
 Assistant 
 Let me break down the results for you:
- Calculated Permeability: 83.33 millidarcies (md)
- Rock Type: Sandstone
- Porosity: 20%
- Reservoir Quality Assessment: Poor reservoir quality

The permeability of 83.33 md indicates a relatively low ability for fluids to flow through the rock. This is considered poor for a reservoir, which typically needs higher permeability for effective hydrocarbon production. Factors like grain size, sorting, and compaction can influence permeability. To improve permeability, you might look for sandstones with larger grain sizes, better sorting, or less compaction.


**Petrophysical Applications:** This Permeability Calculator tool showcases how to implement petroleum engineering formulas into custom tools. Using the Kozeny-Carman equation, it calculates reservoir permeability - a critical property that indicates how easily fluids flow through rock. The tool:

1. Takes parameters a petroleum engineer would know (porosity, grain size, rock type)
2. Applies the appropriate rock-specific constants based on lithology
3. Returns meaningful results with unit conversions (millidarcy) 
4. Provides a qualitative assessment of reservoir quality

For the 20% porosity sandstone with 1mm grain size, the tool calculates a permeability of 83.33 md, which it assesses as "Poor reservoir quality." This type of specialized tool enables petroleum engineers to quickly evaluate formation properties without needing to manually perform complex calculations.

## Lab 4: Custom Tool Response UI Elements

In this lab, we'll create custom UI elements for displaying tool responses in a way that's intuitive for petroleum engineers. We'll focus on rendering the permeability calculator results with a professional, easy-to-read interface.

In [7]:
invokeAndRenderMessages = async (userInputText) => $$.html(
    (async () => {
        
        const result = await displayAnimatedIndicator(
            agent.invoke(
                { messages: [new HumanMessage(userInputText)] }
            )
        );
    
        // Render all messages in the conversation
        const conversationHtml = `
            <div style="font-family: system-ui, -apple-system, sans-serif; max-width: 800px; margin: 0 auto;">
                ${result.messages.map(message => {
                    switch(message.constructor.name) {
                        case 'HumanMessage':
                            return renderHumanMessage(message);
                        case 'AIMessage':
                            return renderAIMessage(message);
                        case 'ToolMessage':
                            switch (message.name) {
                                case 'calculator':
                                    return renderCalculatorToolMessage(message);
                                case 'userInputTool':
                                    return renderUserInputToolMessage(message);
                                case 'permeabilityCalculator':
                                    return renderPermeabilityCalculatorMessage(message);
                                default:
                                    return (`<div><h4>Tool Message from ${message.name}:</h4><pre>${message.content}</pre></div>`)
                            }
                            
                        default:
                            return (`<div><h4>Message:</h4><pre>${JSON.stringify(message, null, 2)}</pre></div>`);
                    }
                }).join('\n')}
            </div>
        `;
        
        return conversationHtml
    })()
)

// Define available tools
tools = [
    // new Calculator,
    // userInputTool
    permeabilityCalculator
];

// Create the React agent
agent = createReactAgent({
    llm,
    tools,
});

invokeAndRenderMessages(`What is the permeability of 20% porosity sandstone with 1mm average grain size?`)

⏳ Processing request... .....
✅ Response received successfully (took 5.2s)


User 
 What is the permeability of 20% porosity sandstone with 1mm average grain size? 
 


 
 Assistant 
 I'll help you calculate the permeability using the permeabilityCalculator tool with the parameters you've specified. 
 
 
 Tool Calls: 
 
 
 Tool: permeabilityCalculator 
 Arguments: {
 "porosity": 0.2,
 "grainSize": 1,
 "rockType": "sandstone"
} 
 
 
 
 
 


 
 
 Permeability Analysis Results
 
 
 
 Permeability 
 83.33 mD 
 
 
 Rock Type 
 sandstone 
 
 
 Porosity 
 20.0% 
 
 
 Assessment 
 
 Poor reservoir quality
 
 
 
 
 

 
 Assistant 
 Based on the calculation, the estimated permeability for the sandstone with the given properties is 83.33 millidarcies (md). 

The tool provides an additional assessment that this represents "Poor reservoir quality" in terms of permeability. This means that while the rock has some ability to transmit fluids, it would not be considered an ideal reservoir rock for efficient oil or gas extraction.

A few key observations:
- Porosity: 20% (0.2)
- Grain Size: 1 mm
- Rock Type: Sandstone
- Calculated Permeability: 83.33 md (considered poor)

Would you like me to explain anything further about this permeability calculation?

The permeability calculator tool's response is rendered using a custom UI component that displays:
1. A clear "Permeability Analysis Results" header
2. Four key metrics in separate cards:
   - Permeability value in millidarcies (mD)
   - Rock Type (sandstone, limestone, etc.)
   - Porosity percentage
   - Reservoir quality assessment
3. Professional styling with:
   - Clean grid layout
   - Clear typography
   - Visual hierarchy
   - Color-coded assessment
   - Proper units display

This custom rendering makes the tool's output much more readable and professional compared to raw JSON output, helping petroleum engineers quickly interpret the results. The layout is designed to match industry software conventions, making it feel familiar and intuitive to users.


## Conclusion and Next Steps

This workshop has introduced you to building AI agents with LangGraph and AWS Amplify, with specific applications for petroleum engineering. As you continue to develop these skills, consider these next steps:

1. **Integrate with real data sources**:
   - Connect to SCADA systems
   - Import well logs and seismic data
   - Access production databases

2. **Develop more specialized tools**:
   - Material balance calculations
   - Nodal analysis tools
   - Economic evaluation functions
   - Geospatial analysis capabilities

3. **Deploy to production**:
   - Set up proper authentication
   - Implement role-based access control
   - Configure monitoring and logging
   - Establish CI/CD pipelines

4. **Extend the UI**:
   - Build mobile-friendly interfaces for field use
   - Create dashboards for production monitoring
   - Develop collaborative workspaces for team analysis

The combination of AI, custom tools, and cloud infrastructure provides petroleum engineers with powerful new capabilities for data analysis, decision support, and knowledge management across the energy sector.